In [1]:
import torch
print("CUDA available:", torch.cuda.is_available())

CUDA available: True


In [2]:
print("Device count:", torch.cuda.device_count())
if torch.cuda.is_available():
    print("GPU:", torch.cuda.get_device_name(0))

Device count: 1
GPU: NVIDIA GeForce RTX 4060 Laptop GPU


In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import PeftModel
import torch, re

In [2]:
BASE_ID = "microsoft/Phi-3-mini-4k-instruct"
ADAPTER_DIR = "phi3mini-agnews-smoke-adapters"  # <- your saved adapters folder

bnb = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype="bfloat16"  # if this errors on your setup, change to "float16"
)

tok = AutoTokenizer.from_pretrained(BASE_ID, use_fast=True)
if tok.pad_token is None:
    tok.pad_token = tok.eos_token

base = AutoModelForCausalLM.from_pretrained(
    BASE_ID,
    quantization_config=bnb,
    device_map="auto"
)

model = PeftModel.from_pretrained(base, ADAPTER_DIR)
model.eval()
print("Loaded on:", model.device)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Loaded on: cuda:0


In [4]:
LABELS = ["World", "Sports", "Business", "Sci/Tech"]

def classify_news(text: str, max_new_tokens: int = 6):
    prompt = (
        "### Instruction:\n"
        "Classify the news into one of: World, Sports, Business, Sci/Tech. "
        "Answer with the label only.\n\n"
        "### Input:\n"
        f"{text.strip()}\n\n"
        "### Response:\n"
    )
    inputs = tok(prompt, return_tensors="pt").to(model.device)

    with torch.inference_mode():
        out = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=False,
            eos_token_id=tok.eos_token_id,
            pad_token_id=tok.pad_token_id,
        )

    full = tok.decode(out[0], skip_special_tokens=True)
    # keep only the first line after "### Response:"
    after = full.split("### Response:")[-1].strip()
    label = after.splitlines()[0].strip()

    # light cleanup: strip punctuation/extra words
    label = re.sub(r"[^A-Za-z/]+", " ", label).strip()
    # optional: map close variants to canonical labels
    canon = min(LABELS, key=lambda L: len(set(L.lower()) - set(label.lower())))
    return label, canon, full


In [5]:
samples = [
    "India's benchmark indices rallied as IT and banking stocks gained after the policy announcement.",
    "NASA confirms the rover discovered hydrated minerals on Mars.",
    "Manchester City secured a 2-0 win with a late brace from Haaland.",
    "Oil prices fell as OPEC signaled potential output increases."
]

for s in samples:
    label, canon, raw = classify_news(s)
    print(f"\nText: {s}\nPred: {label}  |  Canon: {canon}")


c:\Users\akshaysi\anaconda3\envs\deepl_ml_env\Lib\site-packages\transformers\integrations\sdpa_attention.py:89: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(



Text: India's benchmark indices rallied as IT and banking stocks gained after the policy announcement.
Pred: Business  |  Canon: Business

Text: NASA confirms the rover discovered hydrated minerals on Mars.
Pred: Sci/Tech  |  Canon: Sci/Tech

Text: Manchester City secured a 2-0 win with a late brace from Haaland.
Pred: Sports  |  Canon: Sports

Text: Oil prices fell as OPEC signaled potential output increases.
Pred: Business  |  Canon: Business


In [6]:
def ask_one(messages, max_new_tokens=64):
    prompt = tok.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    x = tok(prompt, return_tensors="pt").to(model.device)
    y = model.generate(**x, max_new_tokens=max_new_tokens, do_sample=False,
                       eos_token_id=tok.eos_token_id, pad_token_id=tok.pad_token_id)
    return tok.decode(y[0], skip_special_tokens=True)

ask_one([
    {"role": "system", "content": "You are concise and helpful."},
    {"role": "user", "content": "Classify: 'Indexes rally as banks surge after policy.' Respond with one label: World, Sports, Business, Sci/Tech."}
], max_new_tokens=6)


"You are concise and helpful. Classify: 'Indexes rally as banks surge after policy.' Respond with one label: World, Sports, Business, Sci/Tech. Business"

In [7]:
ask_one(
    [
        {"role": "system", "content": "You are concise and helpful."},
        {"role": "user", "content": "Classify: 'Indexes rally as banks surge after policy.' Respond with one label: World, Sports, Business, Sci/Tech."}
    ],
    max_new_tokens=6
)


"You are concise and helpful. Classify: 'Indexes rally as banks surge after policy.' Respond with one label: World, Sports, Business, Sci/Tech. Business"

In [11]:
import re

LABELS = ["World", "Sports", "Business", "Sci/Tech"]

def _norm(s: str) -> str:
    # normalize for robust matching (case/punct/space insensitive)
    return re.sub(r"[^a-z]", "", s.lower())

def snap_to_label(raw: str) -> str:
    # take only the first line
    line = raw.strip().splitlines()[0]
    n = _norm(line)
    # exact canonical match first
    for L in LABELS:
        if _norm(L) == n:
            return L
    # contains canonical token
    for L in LABELS:
        if _norm(L) in n or n in _norm(L):
            return L
    # last resort: regex search in raw
    m = re.search(r"\b(World|Sports|Business|Sci/?Tech)\b", raw, flags=re.I)
    return m.group(0).replace("SciTech","Sci/Tech") if m else line

def classify_one(text: str, max_new_tokens: int = 4) -> str:
    # Use the same SFT prompt style we trained on and force short output
    prompt = (
        "### Instruction:\n"
        "Classify the news into exactly one of these labels: World, Sports, Business, Sci/Tech.\n"
        "Output the label only—no extra words.\n\n"
        "### Input:\n"
        f"{text.strip()}\n\n"
        "### Response:\n"
    )
    x = tok(prompt, return_tensors="pt").to(model.device)
    y = model.generate(
        **x,
        max_new_tokens=max_new_tokens,
        do_sample=False,
        eos_token_id=tok.eos_token_id,
        pad_token_id=tok.pad_token_id,
    )
    out = tok.decode(y[0], skip_special_tokens=True)
    # Keep only what's after "### Response:" and snap to canonical label
    after = out.split("### Response:")[-1]
    return snap_to_label(after)


In [13]:
classify_one("NASA confirms the rover discovered hydrated minerals on Mars.")

'Sci/Tech'

In [14]:
classify_one("Manchester City won 2-0 with a late brace from Haaland.")

'Sports'

In [15]:
classify_one("Oil prices fell as OPEC signaled potential output increases.")

'Business'

In [16]:
texts = [
    "Indices rallied as banking stocks surged after the policy announcement.",
    "OpenAI releases a new small language model for on-device use.",
    "India defeats Australia in a last-over thriller."
]
[ (t, classify_one(t)) for t in texts ]


[('Indices rallied as banking stocks surged after the policy announcement.',
  'Business'),
 ('OpenAI releases a new small language model for on-device use.', 'Sci/Tech'),
 ('India defeats Australia in a last-over thriller.', 'Sports')]

In [18]:
classify_one("India have to chase 200 runs in one-day match")

'Sports'